In [1]:
# importing module
import sys
import os
from pathlib import Path
sys.path.append(str(Path.cwd().parent))
from myutils import *
from SLBQP import SLBQP
from datastorer import DataStorer
import cvxopt
from cvxopt import matrix
from cvxopt import solvers

In [2]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import time
plt.style.use('seaborn')
params = {
    'figure.figsize': (20, 10),
    'legend.fontsize': 'x-large',
    'axes.labelsize': 'x-large',
    'axes.titlesize':'x-large',
    'xtick.labelsize':'x-large',
    'ytick.labelsize':'x-large',
    'axes.titlesize': 'x-large',
}
plt.rcParams.update(params)

# Parametri default

In [3]:
seed = 123     # random seed
np.random.seed(seed)
ROSEN = 2
GOLDSTEIN = 1

In [4]:
epsilons = [10**i for i in range(-2, 1)]
upper_bounds = [0.1, 1, 5, 10]
problem_sizes = [200,300,400,600]
n_prob = 10

In [4]:
def generate_multiple_alphas(feat, targ, size, aplhas, n=5, startdir='./', maxiter=1000):
    """experiments on the alpha paramerer."""
    if not startdir.endswith("/"):
        print("inserting slash in startdir")
        startdir = startdir + "/"
    
    for alpha in alphas:
        alpha_name = str(alpha).replace(".","")
        os.makedirs(f"{startdir}{alpha_name}", exist_ok=True)
        print(f"creating {startdir}{alpha_name}")
    for i in range(n):
     
        K, target = sample_transform_problem(feat, targ, size)
        
        print("Solving with solver")
        sg, xg, vg, itg = SLBQP(K, target, 1, 0.1, eps=1e-6, maxIter=maxiter, alpha=1, prj_type=GOLDSTEIN)
        with open(f"{startdir}optimal_values.txt", "a") as f:
            f.write(str(vg) + "\n")
            
        for alpha in alphas:
            print(f"Problem {i} {alpha=}")        
            print(f"\tAbout to run GOLDSTEIN with {alpha=}")
            
            ds = DataStorer()
            sg, xg, vg, itg = SLBQP(K, target, 1, 0.1, eps=1e-3, maxIter=maxiter, alpha=alpha, prj_type=GOLDSTEIN, ds=ds)
            
            alpha_name = str(alpha).replace(".","")
            ds.save_to_file(f"{startdir}{alpha_name}/{i}.csv")
            
            print(f"\t\t{itg=}")
            

In [ ]:
def generate_multiple_epsilons(feat, targ, size, u, epsilons, n, startdir='./'):
    u_name = str(u).replace(".","")
    for e in epsilons:
        eps_name = str(e).replace(".","")
        os.makedirs(f"{startdir}C{u_name}EPS{eps_name}/{size}/GOLD", exist_ok=True)
        os.makedirs(f"{startdir}C{u_name}EPS{eps_name}/{size}/ROS", exist_ok=True)

    for i in range(n):
        print(f"generating problem {i}")
        K, target = sample_transform_problem(feat, targ, size)
        for e in epsilons:
            print(f"Solving problem {i} with epsilon {e}")
            eps_name = str(e).replace(".","")
            ds = DataStorer()
            print(f"\tAbout to run GOLDSTEIN with {e=}")
            sg, xg, vg, itg = SLBQP(K, target, u, e, eps=1e-3, maxIter=-1, lmb0=0, d_lmb=2, prj_eps=1e-9, prj_type=GOLDSTEIN, ds=ds)
            ds.save_to_file(f"{startdir}C{u_name}EPS{eps_name}/{size}/GOLD/{i}.csv")
            ds = DataStorer()
            print(f"\tAbout to run ROSEN with {e=}")
            sg, xg, vg, itg = SLBQP(K, target, u, e, eps=1e-3, maxIter=-1, prj_type=ROSEN, ds=ds)
            ds.save_to_file(f"{startdir}C{u_name}EPS{eps_name}/{size}/ROS/{i}.csv")
            print("Solving with higher precision")
            sg, xg, vg, itg = SLBQP(K, target, u, e, eps=1e-6, maxIter=-1, lmb0=0, d_lmb=2, prj_eps=1e-9, prj_type=GOLDSTEIN)
            with open(f"{startdir}C{u_name}EPS{eps_name}/{size}/optimal_values.txt", "a") as f:
                f.write(str(vg) + "\n")
            
            

In [ ]:
def generate_multiple_upperbounds(feat, targ, size, upper_bounds, epsilon ,n, startdir='./'):
    
    eps_name = str(epsilon).replace(".", "")
    for u in upper_bounds:
        u_name = str(u).replace(".","")
        os.makedirs(f"{startdir}C{u_name}EPS{eps_name}/{size}/GOLD", exist_ok=True)
        os.makedirs(f"{startdir}C{u_name}EPS{eps_name}/{size}/ROS", exist_ok=True)
        
    for i in range(n):
        print(f"generating problem {i}")
        K, target = sample_transform_problem(feat, targ, size)  
        for u in upper_bounds:
            print(f"Solving problem {i} with U: {u}")
            u_name = str(u).replace(".","")
            ds = DataStorer()
            print(f"\tAbout to run GOLDSTEIN with {u=}")
            sg, xg, vg, itg = SLBQP(K, target, u, epsilon, eps=1e-3, maxIter=-1, lmb0=0, d_lmb=2, prj_eps=1e-9, prj_type=GOLDSTEIN, ds=ds)
            ds.save_to_file(f"{startdir}C{u_name}EPS{eps_name}/{size}/GOLD/{i}.csv")
            ds = DataStorer()
            print(f"\tAbout to run ROSEN with {u=}")
            sg, xg, vg, itg = SLBQP(K, target, u, epsilon, eps=1e-3, maxIter=-1, prj_type=ROSEN, ds=ds)
            ds.save_to_file(f"{startdir}C{u_name}EPS{eps_name}/{size}/ROS/{i}.csv")
            print("Solving with higher precision")
            sg, xg, vg, itg = SLBQP(K, target, u, epsilon, eps=1e-6, maxIter=-1, lmb0=0, d_lmb=2, prj_eps=1e-9, prj_type=GOLDSTEIN)
            with open(f"{startdir}C{u_name}EPS{eps_name}/{size}/optimal_values.txt", "a") as f:
                f.write(str(vg)+"\n")

In [ ]:
feat, targ = load_ml_dataset()
t1 = targ[:,0]
t2 = targ[:,1]
targ = t1
ROOT="./RUNS/ML/"

In [ ]:
for size in problem_sizes:
    generate_multiple_epsilons(feat, targ, size, 1, epsilons, n_prob, startdir=ROOT)

In [ ]:
for size in problem_sizes:
    generate_multiple_upperbounds(feat, targ, size, upper_bounds, 0.1, n_prob, startdir=ROOT)

In [ ]:
feat, targ = load_california_dataset()
targ
ROOT="./RUNS/CAL/"

In [ ]:
for size in problem_sizes:
    generate_multiple_epsilons(feat, targ, size, 1, epsilons, n_prob, startdir=ROOT)

In [ ]:
for size in problem_sizes:
    generate_multiple_upperbounds(feat, targ, size, upper_bounds, 0.1, n_prob, startdir=ROOT)

In [ ]:
feat, targ = load_airfoil_dataset()
targ
ROOT="./RUNS/AIRF/"

In [ ]:
for size in problem_sizes:
    generate_multiple_epsilons(feat, targ, size, 1, epsilons, n_prob, startdir=ROOT)

In [ ]:
for size in problem_sizes:
    generate_multiple_upperbounds(feat, targ, size, upper_bounds, 0.1, n_prob, startdir=ROOT)

# ALPHA EXPERIMENTS

In [5]:
alphas = [0.1, 0.5, 1, 2, 5]

In [6]:
# ML DATASET
startdir = f"RUNS/ML/ALPHA/"
feat, targ = load_ml_dataset()
targ = targ[:,0]
size = 500
generate_multiple_alphas(feat, targ, size, alphas, n=5, startdir=startdir, maxiter=1000)

loading from: 
/Users/gerardozinno/repos/ProgettoML/cm/data/ML-CUP19-TR.csv
creating RUNS/ML/ALPHA/01
creating RUNS/ML/ALPHA/05
creating RUNS/ML/ALPHA/1
creating RUNS/ML/ALPHA/2
creating RUNS/ML/ALPHA/5
Solving with solver
Problem 0 alpha=0.1
	About to run GOLDSTEIN with alpha=0.1
		itg=124
Problem 0 alpha=0.5
	About to run GOLDSTEIN with alpha=0.5
		itg=112
Problem 0 alpha=1
	About to run GOLDSTEIN with alpha=1
		itg=225
Problem 0 alpha=2
	About to run GOLDSTEIN with alpha=2
		itg=1000
Problem 0 alpha=5
	About to run GOLDSTEIN with alpha=5
		itg=1000
Solving with solver
Problem 1 alpha=0.1
	About to run GOLDSTEIN with alpha=0.1
		itg=135
Problem 1 alpha=0.5
	About to run GOLDSTEIN with alpha=0.5
		itg=72
Problem 1 alpha=1
	About to run GOLDSTEIN with alpha=1
		itg=134
Problem 1 alpha=2
	About to run GOLDSTEIN with alpha=2
		itg=1000
Problem 1 alpha=5
	About to run GOLDSTEIN with alpha=5
		itg=1000
Solving with solver
Problem 2 alpha=0.1
	About to run GOLDSTEIN with alpha=0.1
		itg=143

In [7]:
# CAL DATASET
startdir = f"./RUNS/CAL/ALPHA/"
feat, targ = load_california_dataset()
size = 500
generate_multiple_alphas(feat, targ, size, alphas, n=5, startdir=startdir, maxiter=1000)

/usr/local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.datasets.california_housing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


creating ./RUNS/CAL/ALPHA/01
creating ./RUNS/CAL/ALPHA/05
creating ./RUNS/CAL/ALPHA/1
creating ./RUNS/CAL/ALPHA/2
creating ./RUNS/CAL/ALPHA/5
Solving with solver
Problem 0 alpha=0.1
	About to run GOLDSTEIN with alpha=0.1
		itg=91
Problem 0 alpha=0.5
	About to run GOLDSTEIN with alpha=0.5
		itg=20
Problem 0 alpha=1
	About to run GOLDSTEIN with alpha=1
		itg=8
Problem 0 alpha=2
	About to run GOLDSTEIN with alpha=2
		itg=1000
Problem 0 alpha=5
	About to run GOLDSTEIN with alpha=5
		itg=1000
Solving with solver
Problem 1 alpha=0.1
	About to run GOLDSTEIN with alpha=0.1
		itg=91
Problem 1 alpha=0.5
	About to run GOLDSTEIN with alpha=0.5
		itg=20
Problem 1 alpha=1
	About to run GOLDSTEIN with alpha=1
		itg=8
Problem 1 alpha=2
	About to run GOLDSTEIN with alpha=2
		itg=1000
Problem 1 alpha=5
	About to run GOLDSTEIN with alpha=5
		itg=1000
Solving with solver
Problem 2 alpha=0.1
	About to run GOLDSTEIN with alpha=0.1
		itg=91
Problem 2 alpha=0.5
	About to run GOLDSTEIN with alpha=0.5
		itg=20


In [8]:
startdir = f"RUNS/AIRF/ALPHA/"
feat, targ= load_airfoil_dataset()
size = 500
generate_multiple_alphas(feat, targ, size, alphas, n=5, startdir=startdir, maxiter=1000)

/Users/gerardozinno/repos/ProgettoML/cm/data/airfoil_self_noise.csv
creating RUNS/AIRF/ALPHA/01
creating RUNS/AIRF/ALPHA/05
creating RUNS/AIRF/ALPHA/1
creating RUNS/AIRF/ALPHA/2
creating RUNS/AIRF/ALPHA/5
Solving with solver
Problem 0 alpha=0.1
	About to run GOLDSTEIN with alpha=0.1
		itg=73
Problem 0 alpha=0.5
	About to run GOLDSTEIN with alpha=0.5
		itg=22
Problem 0 alpha=1
	About to run GOLDSTEIN with alpha=1
		itg=23
Problem 0 alpha=2
	About to run GOLDSTEIN with alpha=2
		itg=1000
Problem 0 alpha=5
	About to run GOLDSTEIN with alpha=5
		itg=1000
Solving with solver
Problem 1 alpha=0.1
	About to run GOLDSTEIN with alpha=0.1
		itg=77
Problem 1 alpha=0.5
	About to run GOLDSTEIN with alpha=0.5
		itg=19
Problem 1 alpha=1
	About to run GOLDSTEIN with alpha=1
		itg=11
Problem 1 alpha=2
	About to run GOLDSTEIN with alpha=2
		itg=1000
Problem 1 alpha=5
	About to run GOLDSTEIN with alpha=5
		itg=1000
Solving with solver
Problem 2 alpha=0.1
	About to run GOLDSTEIN with alpha=0.1
		itg=71
Pro

array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
          37.88      , -122.23      ],
       [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
          37.86      , -122.22      ],
       [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
          39.43      , -121.32      ],
       [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
          39.37      , -121.24      ]])

In [11]:
feat

array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894])